In [1]:
import time
import datetime
from mkndaq.inst.neph import NEPH
import schedule
import threading

cfg = {'ne300': {
                'type': 'NE300',
                'serial_number': '23-0690',
                'serial_id': 0,
                'protocol': 'acoem',
                'socket': {
                    'host': '192.168.0.50',
                    'port': 32783,
                    'timeout': 10,
                },
                'datalog': {
                    'parameters': [2635000, 2525000, 2450000, 2635090, 2525090, 2450090, 
                                   5001, 5002, 5003, 5004, 5005, 5006, 5010, 
                                   26635000, 26525000, 26450000, 
                                   13525000, 
                                   15635000, 15525000, 15450000, 
                                   11635000, 11525000, 11450000, 11635090, 11525090, 11450090, 
                                   6007, 6008, 6001, 6002, 6003, 
                                   6635000, 6525000, 6450000, 6635090, 6525090, 6450090, 
                                   5011, 5012, 5014, 4035],
                    'interval': 60, 
                },
                'get_data_interval': 3, # minutes. logger retrieval interval
                'zero_span_check_interval': 13,
                'zero_check_duration': 5,
                'span_check_duration': 3,
                'staging_zip': True,  
                'verbosity': 2,  # 0: silent, 1: medium, 2: full          
            },
            'reporting_interval': 10,
            'logs': 'mkndaq/logs',
            'data': 'mkndaq/data',
            'staging': {
                'path': 'mkndaq/staging',
            },
}
ne300 = NEPH('ne300', cfg, verbosity=0)
# # Initialize NEPH (name: ne300  S/N: 23-0690)
#   Instrument identified itself as '{'Model': 158, 'Variant': 300, 'Sub-Type': 0, 'Range': 0, 'Build': 158, 'Branch': 300}'.

# Initialize NEPH (name: ne300  S/N: 23-0690)
  - Instrument identified itself as '{'id': 'ACOEM Aurora NE-300 Sub-Type: 0 Range: 0 Build: 158 Branch: 300'}'.
  - Instrument current operation: ambient.
2024-05-30 17:26:49 [ne300] dtm found: 2024-05-30 17:26:48 > dtm after set: 2024-05-30 17:26:48.
  - Currently logged parameters: [41, 2635000, 2525000, 2450000, 2635090, 2525090, 2450090, 5001, 5002, 5003, 5004, 5005, 5006, 5010, 26635000, 26525000, 26450000, 13525000, 15635000, 15525000, 15450000, 11635000, 11525000, 11450000, 11635090, 11525090, 11450090, 6007, 6008, 6001, 6002, 6003, 6635000, 6525000, 6450000, 6635090, 6525090, 6450090, 5011, 5012, 5014, 4035].


In [2]:
def run_threaded(job_func):
    """Set up threading and start job.

    Args:
        job_func ([type]): [description]
    """
    job_thread = threading.Thread(target=job_func)
    job_thread.start()

# align start with a 10' timestamp
while int(time.time()) % 10 > 0:
    time.sleep(0.1)

fetch = 20
schedule.every(fetch).seconds.do(ne300.print_ssp_bssp)
schedule.every(cfg['ne300']['get_data_interval']).minutes.at(':10').do(run_threaded, ne300.get_new_data)
schedule.every(cfg['ne300']['zero_span_check_interval']).minutes.at(':00').do(run_threaded, ne300.do_zero_span_check)

while True:
    schedule.run_pending()
    time.sleep(1)

2024-05-30 17:27:20 [ne300] ssp|bssp (Mm-1) r: -0.3532|-0.6674 g: 0.6819|0.1931 b: 0.6582|0.1712
2024-05-30 17:27:40 [ne300] ssp|bssp (Mm-1) r: -0.3493|-0.6658 g: 0.6844|0.1905 b: 0.6505|0.1683
2024-05-30 17:28:00 [ne300] ssp|bssp (Mm-1) r: -0.3352|-0.6598 g: 0.6849|0.1911 b: 0.6470|0.1758
2024-05-30 17:28:20 [ne300] ssp|bssp (Mm-1) r: -0.3166|-0.6663 g: 0.6723|0.1873 b: 0.6368|0.1773
2024-05-30 17:28:40 [ne300] ssp|bssp (Mm-1) r: -0.3243|-0.6682 g: 0.6609|0.1874 b: 0.6310|0.1784
2024-05-30 17:29:00 [ne300] ssp|bssp (Mm-1) r: -0.3276|-0.6666 g: 0.6637|0.1841 b: 0.6239|0.1712
2024-05-30 17:29:20 [ne300] ssp|bssp (Mm-1) r: -0.3401|-0.6659 g: 0.6514|0.1880 b: 0.6249|0.1741
2024-05-30 17:29:40 [ne300] ssp|bssp (Mm-1) r: -0.3397|-0.6655 g: 0.6470|0.1862 b: 0.6240|0.1712
2024-05-30 17:30:00 .get_new_data (name=ne300, save=True)
2024-05-30 17:30:00 [ne300] ssp|bssp (Mm-1) r: -0.3360|-0.6680 g: 0.6486|0.1850 b: 0.6294|0.1702
2024-05-30 17:30:20 [ne300] ssp|bssp (Mm-1) r: -0.3449|-0.6705 g: 0.6

KeyboardInterrupt: 

In [ ]:
res = ne300.get_id()
print(res)

In [ ]:
# mimick VI099 response
ne300.get_current_data(strict=True, verbosity=0)

In [ ]:
ne300.get_current_data(verbosity=0)

In [ ]:
ne300.get_values([1635000, 1525000], verbosity=2)

In [ ]:
# retrieve VI099 command parameters
ne300.get_values(parameters=[1, 1635000,1525000,1450000,1635090,1525090,1450090,5001,5004,5003,5002,4036,4035])

In [ ]:
ne300.get_current_data(add_params=[1001, 1002, 1003, 1004, 1005, 2001, 2002])

In [ ]:
# not implemented
ne300.get_new_data()

In [ ]:
# not implemented
ne300.get_all_data()

In [ ]:
ne300.get_instr_type()

In [ ]:
ne300.get_datetime()

In [ ]:
ne300.get_datalog_config()

In [ ]:
# cycle through operating states
# 0: ambient, 1: zero, 2: span
for i in [1, 2, 0]:
    print(f"current: {ne300.get_current_operation()} > set: {i} ...")
    print(f"done: {ne300.set_current_operation(state=i)} okay")
    time.sleep(3)

In [ ]:
from datetime import datetime, timezone, timedelta
end = datetime.now(timezone.utc)
start = end - timedelta(minutes=10)
print(f"start: {start}\nend  : {end}")
data = ne300.get_logged_data(start=start, end=end, verbosity=1)
# for i in range(len(data)):
    # print(data[i])

In [ ]:
# repeat packet
message = ne300._acoem_construct_message(command=7, payload=bytes([0,0,0,1]))
response = ne300.tcpip_comm(message=message)
print(response)
display(ne300._acoem_decode_logged_data(response=response))

In [ ]:
# next packet
message = ne300._acoem_construct_message(command=7, payload=bytes([0,0,0,0]))
response = ne300.tcpip_comm(message=message)
response